<a href="https://colab.research.google.com/github/pt-kunal-mishra/deep_learning_ml/blob/main/resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Face recogination using Pytorch***

In [2]:
!pip install facenet-pytorch

In [1]:
import torch
from facenet_pytorch import MTCNN,InceptionResnetV1
mtcnn=MTCNN(image_size=250,margin=0,min_face_size=10)
resnet1=InceptionResnetV1(pretrained='vggface2').eval()


  0%|          | 0.00/107M [00:00<?, ?B/s]

In [3]:
help(MTCNN)

Help on class MTCNN in module facenet_pytorch.models.mtcnn:

class MTCNN(torch.nn.modules.module.Module)
 |  MTCNN(image_size=160, margin=0, min_face_size=20, thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True, select_largest=True, selection_method=None, keep_all=False, device=None)
 |  
 |  MTCNN face detection module.
 |  
 |  This class loads pretrained P-, R-, and O-nets and returns images cropped to include the face
 |  only, given raw input images of one of the following types:
 |      - PIL image or list of PIL images
 |      - numpy.ndarray (uint8) representing either a single image (3D) or a batch of images (4D).
 |  Cropped faces can optionally be saved to file
 |  also.
 |  
 |  Keyword Arguments:
 |      image_size {int} -- Output image size in pixels. The image will be square. (default: {160})
 |      margin {int} -- Margin to add to bounding box, in terms of pixels in the final image. 
 |          Note that the application of the margin differs slightly from the 

In [7]:
from pickle import TRUE
from PIL import Image
img=Image.open("/content/photo.jpg")
img_cropped=mtcnn(img,save_path='saved/detect.png')
img_embedding=resnet1(img_cropped.unsqueeze(0))
resnet1.classify=True
img_probs=resnet1(img_cropped.unsqueeze(0))

In [8]:
from facenet_pytorch import InceptionResnetV1
model=InceptionResnetV1(pretrained="vggface2").eval()
model=InceptionResnetV1(pretrained="casia-webface").eval()
model=InceptionResnetV1(num_classes=100).eval()
model=InceptionResnetV1(classify=True,num_classes=1001).eval()

In [9]:
from facenet_pytorch import MTCNN,InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import os

In [10]:
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [11]:
workers=0 if os.name=='nt' else 4

In [12]:
mtcnn=MTCNN(image_size=160,margin=0,min_face_size=20,thresholds=[0.6,0.7,0.7],factor=0.709,post_process=True,device=device)

In [13]:
resnet=InceptionResnetV1(pretrained="vggface2").eval().to(device)

In [14]:
!sudo apt-get install unzip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [15]:
!unzip test_images.zip

Archive:  test_images.zip
   creating: test_images/angelina_jolie/
  inflating: test_images/angelina_jolie/1.jpg  
   creating: test_images/bradley_cooper/
  inflating: test_images/bradley_cooper/1.jpg  
   creating: test_images/kate_siegel/
  inflating: test_images/kate_siegel/1.jpg  
   creating: test_images/paul_rudd/
  inflating: test_images/paul_rudd/1.jpg  
   creating: test_images/shea_whigham/
  inflating: test_images/shea_whigham/1.jpg  


In [16]:
def zip(x):
  return x[0]
dataset=datasets.ImageFolder("../content/test_images")
dataset.idx_to_class={i:c for c,i in dataset.class_to_idx.items()}
loader=DataLoader(dataset,collate_fn=zip,num_workers=workers)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [17]:
aligned=[]
names=[]
for x,y in loader:
  x_aligned,prob=mtcnn(x,return_prob=True)
  if x_aligned is not None:
    print("face Detected with probablility:{:8f}".format(prob))
    aligned.append(x_aligned)
    names.append(dataset.idx_to_class[y])

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


face Detected with probablility:0.999983
face Detected with probablility:0.999934
face Detected with probablility:0.999733
face Detected with probablility:0.999876
face Detected with probablility:0.999992


In [18]:
aligned=torch.stack(aligned).to(device)
embeddings=resnet(aligned).detach().cpu()

In [20]:
import pandas as pd


**Print distance matrix for classes**





In [21]:
dists=[[(e1-e2).norm().item() for e2 in embeddings] for e1 in embeddings]
print(pd.DataFrame(dists,columns=names,index=names))

                angelina_jolie  bradley_cooper  kate_siegel  paul_rudd  \
angelina_jolie        0.000000        1.447480     0.887728   1.434377   
bradley_cooper        1.447480        0.000000     1.313749   1.011981   
kate_siegel           0.887728        1.313749     0.000000   1.388992   
paul_rudd             1.434377        1.011981     1.388992   0.000000   
shea_whigham          1.399073        1.038684     1.379655   1.104095   

                shea_whigham  
angelina_jolie      1.399073  
bradley_cooper      1.038684  
kate_siegel         1.379655  
paul_rudd           1.104095  
shea_whigham        0.000000  
